# Practica 1

## Instalación de dependencias

In [ ]:
!pip install -q bitsandbytes datasets accelerate peft transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 17.5 MB/s eta 0:00:00


In [ ]:
!pip install -q safetensors

## Importaciones

In [ ]:
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)

## Preparación del dataset

In [ ]:
# Cargar dataset (cambia aquí para tu propio dataset)
data = load_dataset("glue", "sst2")

def merge_columns(example):
    label = "POSITIVE" if example["label"] == 1 else "NEGATIVE"
    example['prediction'] = example["sentence"] + " ->: " + label
    return example

data['train'] = data['train'].map(merge_columns)

# Para experimentar: Si subes tu CSV, reemplaza lo de arriba con:
# from google.colab import files
# uploaded = files.upload()
# import pandas as pd
# from datasets import Dataset
# df = pd.read_csv(list(uploaded.keys())[0])  # Asume columnas 'text' y 'output'
# data = {'train': Dataset.from_pandas(df.rename(columns={'text': 'sentence', 'output': 'label'}))}
# # Luego aplica merge_columns como arriba

print("Ejemplo de datos formateados:")
print(data['train'][0]['prediction'])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sst2/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

sst2/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

sst2/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Ejemplo de datos formateados:
hide new secretions from the parental units  ->: NEGATIVE


## Carga del modelo y tokenizer

In [ ]:
model_name = "deepseek-ai/deepseek-coder-1.3b-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map='auto',
)

# Configuraciones básicas
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


## Congelar parámetros del modelo

In [ ]:
for param in model.parameters():
    param.requires_grad = False
    if param.ndim == 1:
        param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)

model.lm_head = CastOutputToFloat(model.lm_head)

## Verificar parámetros entrenables

In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")

print_trainable_parameters(model)

trainable params: 0 || all params: 1346471936 || trainable%: 0.0


## Identificar módulos objetivo para LoRA

In [ ]:
for name, module in model.named_modules():
    if 'attn' in name.lower() or 'attention' in name.lower():
        print(name)
        for sub_name, sub_module in module.named_modules():
            if 'proj' in sub_name:
                print(f"  - {sub_name}")

model.layers.0.self_attn
  - q_proj
  - k_proj
  - v_proj
  - o_proj
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.post_attention_layernorm
model.layers.1.self_attn
  - q_proj
  - k_proj
  - v_proj
  - o_proj
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.post_attention_layernorm
model.layers.2.self_attn
  - q_proj
  - k_proj
  - v_proj
  - o_proj
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.self_attn.o_proj
model.layers.2.post_attention_layernorm
model.layers.3.self_attn
  - q_proj
  - k_proj
  - v_proj
  - o_proj
model.layers.3.self_attn.q_proj
model.layers.3.self_attn.k_proj
model.layers.3.self_attn.v_proj
model.layers.3.self_attn.o_proj
model.layers.3.post_attention_layernorm
model.layers.4.self_attn
  - q_proj
  - k_proj
  - v

## Setup de LoRA

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

trainable params: 3145728 || all params: 1349617664 || trainable%: 0.2330829007288393


## Tokenización del dataset

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['prediction'], truncation=True, padding='max_length', max_length=128)

tokenized_data = data['train'].map(tokenize_function, batched=True, remove_columns=data['train'].column_names)

tokenized_data.set_format("torch", columns=["input_ids", "attention_mask"], dtype=torch.long)

print("Ejemplo de datos tokenizados (dtype):", tokenized_data[0]['input_ids'].dtype)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Ejemplo de datos tokenizados (dtype): torch.int64


## Setup del entrenamiento

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    warmup_steps=10,
    max_steps=50,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    output_dir='outputs',
    optim="paged_adamw_8bit",
    save_steps=50,
    report_to="none",
)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

model.config.use_cache = False

trainer = Trainer(
    model=model,
    train_dataset=tokenized_data,
    args=training_args,
    data_collator=data_collator,
)

## Entrenamiento

In [ ]:
trainer.train()

## Guardar el modelo LoRA

In [ ]:
model.save_pretrained("./lora_model")
tokenizer.save_pretrained("./lora_model")
print("Modelo LoRA guardado en ./lora_model")

## Inferencia simple (evaluación)

In [ ]:
# Cargar modelo base + LoRA guardado
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map='auto',
)
model = PeftModel.from_pretrained(base_model, "./lora_model")
tokenizer = AutoTokenizer.from_pretrained("./lora_model")

# Prompt de prueba (similar al formato de entrenamiento)
prompt = "This film was excellent ->: "
inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

# Generar
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=10, do_sample=True, temperature=0.7)

prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Predicción:", prediction)